In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
import jieba
import jieba.analyse

import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split  

def cuttrain_data(df):
  cuttitle = []
  for d in df['title'].astype(str):
    cut = jieba.analyse.extract_tags(d,topK=2)
    cuttitle.append(cut)
  print("in cuttitle")
  # for d in cuttitle:
  #   print(d)
  cutcontext = []
  for d in df['context']:
    cut = jieba.analyse.extract_tags(d,topK=2)
    cutcontext.append(cut)
  # print("in context")
  # for d in cutcontext:
  #   print(d)
  push = []
  for d in df['push']:
    push.append(d)
  down = []
  for d in df['down']:
    down.append(d)

  data = {'title':cuttitle ,'context':cutcontext ,'push':push ,'down':down}
  df1 = pd.DataFrame(data)

  df1['context'] = df1['context'].apply(lambda x : str(x)[1:-2].replace("'","").replace(",",""))
  # print("df1['context'] = ")
  # print(df1['context'])
  df1['title'] = df1['title'].apply(lambda x : str(x)[1:-2].replace("'","").replace(",",""))
  # print("df1['title'] = ")
  # print(df1['title'])
  df1['ans_push'] = df1['title'].astype(str)+' '+df1['context'].astype(str)+' '+df1['push'].astype(str)
  # print("df1['ans_push'] = ")
  # for d in df1['ans_push']:
  #   print(d)
  df1['ans_down'] = df1['title'].astype(str)+' '+df1['context'].astype(str)+' '+df1['down'].astype(str)
  # print("df1['ans_down'] = ")
  # for d in df1['ans_down']:
  #   print(d)
  return df1

def cuttest_data(df):
  cuttitle = []
  for d in df['title'].astype(str):
    cut = jieba.analyse.extract_tags(d,topK=2)
    cuttitle.append(cut)
  cutcontext = []
  for d in df['context']:
    cut = jieba.analyse.extract_tags(d,topK=10)
    cutcontext.append(cut)

  data = {'title':cuttitle ,'context':cutcontext}
  df1 = pd.DataFrame(data)

  df1['context'] = df1['context'].apply(lambda x : str(x)[1:-2].replace("'","").replace(",",""))
  df1['title'] = df1['title'].apply(lambda x : str(x)[1:-2].replace("'","").replace(",",""))
  return df1

def count_push(df):
  vectorizer = CountVectorizer()
  X = vectorizer.fit_transform(df['ans_push'])
  tfidf = TfidfTransformer() 
  tf=tfidf.fit_transform(X)
  word = vectorizer.get_feature_names() #詞袋 
  return tf

def count_down(df):
  vectorizer = CountVectorizer()
  X = vectorizer.fit_transform(df['ans_down'])
  tfidf = TfidfTransformer() 
  tf=tfidf.fit_transform(X)
  word = vectorizer.get_feature_names() #詞袋 
  return tf

def knn_push(x_data,y_data):
  X_train, X_test, y_train, y_test = train_test_split(x_data, y_data['push'].astype(str), random_state=2)
  clf=KNeighborsClassifier(n_neighbors=5)
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  ans_push = y_pred    # 預測模型
  print("y_pred")
  print(y_pred)
  
  y_test=y_test.values
  # print("精準度：",metrics.accuracy_score(y_test, y_pred))
  return ans_push
def knn_down(x_data,y_data):
  X_train, X_test, y_train, y_test = train_test_split(x_data, y_data['down'].astype(str), test_size=0.2, random_state=2)
  clf=KNeighborsClassifier(n_neighbors=5)
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)    # 預測模型
  print("y_pred")
  ans_down = y_pred
 
  y_test=y_test.values
  # print("精準度：",metrics.accuracy_score(y_test, y_pred))
  return ans_down

train_data = pd.read_csv("/content/drive/MyDrive/NatureLanguageProgram/NLP_final_project/train_data.csv")
train_label = pd.read_csv("/content/drive/MyDrive/NatureLanguageProgram/NLP_final_project/train_label.csv")
test_data = pd.read_csv("/content/drive/MyDrive/NatureLanguageProgram/NLP_final_project/test_data.csv")
files = glob('/content/drive/MyDrive/NatureLanguageProgram/NLP_final_project/train*.csv')
df_list = [pd.read_csv(file) for file in files]  #串列中包含兩個Pandas DataFrame
total_train_data = pd.merge(df_list[0], df_list[1], on='index')


the_training_data = total_train_data[['title' ,'context' ,'push' ,'down']]
the_test_data = test_data[['title' ,'context']]


train = cuttrain_data(the_training_data)

test = cuttest_data(the_test_data)

tf_push = count_push(train)
tf_down = count_down(train)

# knn(tf_push,train,test)
# knn(tf_down,train,test)
knn_push(tf_push,train)
knn_down(tf_down,train)

ans_push = knn_push(tf_push,train)
for i in ans_push:
  print(i)
ans_down = knn_down(tf_down,train)
for i in ans_down:
  print(i)


in cuttitle
